In [2]:

from dotenv import load_dotenv
from langchain_teddynote import logging
logging.langsmith("allforone")

LangSmith 추적을 시작합니다.
[프로젝트명]
allforone


In [ ]:
from langgraph.graph import StateGraph, START, END
from agents.state.analysis_state import LocationInsightState
from agents.state.start_state import StartInput
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, HumanMessage, ToolMessage
from utils.util import get_today_str
from utils.llm import LLMProfile
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from prompts import PromptManager, PromptType
from langgraph.prebuilt import ToolNode
from tools.kostat_api import get_move_population, system_prompt
import json
import asyncio

@tool(parse_docstring=True)
def think_tool(reflection: str) -> str:
    """
    [역할]
    당신은 입지/호재 정리 전문가의 내부 반성·점검(Reflection) 담당자입니다.
    최종 보고서에 들어갈 본문(Markdown)을 쓰기 직전에, 데이터 품질·핵심 수치·리스크·보고서용 한 줄 메시지를 짧고 구조적으로 요약해 think_tool에 기록합니다. 이 반성문은 내부용이며, 최종 보고서에 직접 노출되지 않습니다.

    [언제 호출할 것인지]
    - 데이터 수집/정제 → 핵심 수치 산출 → 시계열 해석을 마친 직후 1회 호출(필수)
    - 추가 데이터로 최신 데이터로 바뀌면 갱신 시마다 1회 재호출(선택)

    [강력 지시]
    - 해당 지역에 관련된 내용만 기록
    - 허상 가정,출처 수치 금지
    - 다음 단계(보고서 에이전트)가 바로 쓸 수 있는 한 줄 핵심 메시지 포함

    [나쁜 예]
    - “경제가 좋아진듯함. 분위기 좋음.”(수치·기간·단위·근거 없음)
    - “인근 해운대의 입지지는 이렇다~”(대상 지역 외 서술)
    - “향후 집값 상승 확실.”(근거 없는 단정)

    [검증 체크리스트]
    - 정량 수치가 어긋난 것 이 있는가?
    - GPT가 시계열 판단하기에 좋은 형식으로 되어있는가?
    - 잘못된 내용은 없는가?
    """
    return f"Reflection recorded: {reflection}"

output_key = LocationInsightState.KEY.location_insight_output
start_input_key = LocationInsightState.KEY.start_input
rag_context_key = LocationInsightState.KEY.rag_context
messages_key = LocationInsightState.KEY.messages
target_area_key = StartInput.KEY.target_area
main_type_key = StartInput.KEY.main_type
total_units_key = StartInput.KEY.total_units
web_context_key = LocationInsightState.KEY.web_context
kakao_api_distance_context_key = LocationInsightState.KEY.kakao_api_distance_context


llm = LLMProfile.analysis_llm()
tool_list = [think_tool]
llm_with_tools = llm.bind_tools(tool_list)
tool_node = ToolNode(tool_list)

from tools.gemini_search_tool import gemini_search


def gemini_search_tool(state: LocationInsightState) -> LocationInsightState:
    start_input = state[start_input_key]
    target_area = start_input[target_area_key]
    target_main_type = start_input[main_type_key]
    total_units = start_input[total_units_key]

    prompt = f"""
    <CONTEXT>
    주소: {target_area}
    규모: {total_units}세대
    타입: {target_main_type}
    </CONTEXT>
    <GOAL>
    <CONTEXT> 주변 분양호재를 <OUTPUT>을 참조해서 json 형식으로 출력해주세요
    </GOAL>
    <OUTPUT>
    {{
      "분양호재": [
        {{
          "name": "",
          "location": "",
          "description": "",
          "status": ""
        }}
      ]
    }}
    </OUTPUT>
    """
    result = gemini_search(prompt)

    return {**state, web_context_key: result}

from tools.kakao_api_distance_tool import get_location_profile
def kakao_api_distance_tool(stat: LocationInsightState) -> LocationInsightState:
    start_input = stat[start_input_key]
    target_area = start_input[target_area_key]
    result = get_location_profile(target_area)
    return {**state, kakao_api_distance_context_key: result}

def analysis_setting(state: LocationInsightState) -> LocationInsightState:
    start_input = state[start_input_key]
    target_area = start_input[target_area_key]
    total_units = start_input[total_units_key]
    main_type = start_input[main_type_key]
    rag_context = state[rag_context_key]
    web_context = state[web_context_key]
    kakao_api_distance_context = state[kakao_api_distance_context_key]

    system_prompt = PromptManager(PromptType.LOCATION_INSIGHT_SYSTEM).get_prompt()
    human_prompt = PromptManager(PromptType.LOCATION_INSIGHT_HUMAN).get_prompt(
        target_area=target_area,
        total_units=total_units,
        main_type=main_type,
        date=get_today_str(),
        web_context=web_context,
        rag_context=rag_context,
        kakao_api_distance_context=kakao_api_distance_context,
    ) 
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=human_prompt)
    ]
    return {**state, messages_key: messages}

def agent(state: LocationInsightState) -> LocationInsightState:
    me